In [1]:
from REN import Model
import numpy as np
import cPickle
import theano

/usr/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [8]:
train_data = np.load('Data/Train/qa1_single-supporting-fact_train.npz')
test_data = extract_stories(np.load('Data/Test/qa1_single-supporting-fact_test.npz'))

In [9]:
def extract_stories(data):
    return data['stories'], data['queries'], data['indices'], data['answers']

In [10]:
params = {'embeding_dimension': 100,
          'num_slots': 20,
          'init_learning_rate': 0.01,
          'num_epochs': 20,
          'vocab_size': 160,
          'batch_size': 32,
          'max_sent_len': 7}

params['max_sent_len'] = max(np.shape(train_data['stories'])[2],
                                 np.shape(train_data['queries'])[2])

Ent_Net = Model.EntityNetwork(params['embeding_dimension'],
                              params['vocab_size'],
                              params['num_slots'],
                              params['max_sent_len'])

In [11]:
test_loss, test_accuracy = Ent_Net.test_network(*test_data)
test_accuracy

array(0.0)

In [18]:
f = open('Results/model2.save')
trained = cPickle.load(f)
f.close()

In [19]:
Ent_Net._create_network(trained)

In [20]:
test_loss, test_accuracy = Ent_Net.test_network(*test_data)
test_accuracy

array(0.183)

In [10]:
import numpy as np
from REN import Model
from six.moves import cPickle
import time

params = {'embeding_dimension': 100,
          'num_slots': 20,
          'init_learning_rate': 0.01,
          'num_epochs': 1,
          'vocab_size': 160,
          'batch_size': 32}


def train(path_to_train, path_to_test, params):

    train_data = np.load(path_to_train)  # npz file obj
    test_data = extract_stories(np.load(path_to_test))


    params['max_sent_len'] = max(np.shape(train_data['stories'])[2],
                                 np.shape(train_data['queries'])[2])

    Ent_Net = Model.EntityNetwork(params['embeding_dimension'],
                                  params['vocab_size'],
                                  params['num_slots'],
                                  params['max_sent_len'])

    loss = 0.0
    print('loss training_accuracy       test_loss   test_accuracy')
    for i in range(params['num_epochs']):
        for n,batch in enumerate(get_batch(train_data, params['batch_size'])):
            batch_loss, accuracy = Ent_Net.train_batch(*batch)
            loss = loss + (batch_loss - loss)/(params['batch_size']*1.0)
            if n%10 ==0:
                test_loss, test_accuracy = Ent_Net.test_network(*test_data)
                print('{}   {}  {}  {}').format(batch_loss, accuracy,
                                                test_loss, test_accuracy)


    f = open('Results/model4.save', 'wb')
    cPickle.dump(Ent_Net.params, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()
    
    return Ent_Net, test_data

def get_batch(data, batch_size):
    stories, queries, indices , answers = extract_stories(data)
    N = np.shape(stories)[0]
    all_indices = np.random.permutation(N)
    for i in range(N/batch_size):
        batch_ind = all_indices[i:i + batch_size]
        yield stories[batch_ind], queries[batch_ind], indices[batch_ind] , answers[batch_ind]

def extract_stories(data):
    return data['stories'], data['queries'], data['indices'], data['answers']




In [11]:
Ent_Net, test_data = train('Data/Train/qa1_single-supporting-fact_train.npz',
          'Data/Test/qa1_single-supporting-fact_test.npz', params)

loss training_accuracy       test_loss   test_accuracy
5.17013618421   0.0  5.05509395707  0.0
4.07133330685   0.14375  3.9839906392  0.181
3.06549130317   0.15625  2.9850863593  0.196
2.26393610031   0.15625  2.21094533498  0.209
1.89158359536   0.21875  1.90355492946  0.237
1.7714213035   0.375  1.80813878324  0.347
1.68670251151   0.45  1.70544765379  0.46


In [12]:
test_loss, test_accuracy = Ent_Net.test_network(*test_data)

In [13]:
test_accuracy

array(0.46)